In [1]:
import pandas as pd
import urllib.request as req
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [2]:
req.urlretrieve("https://github.com/e9t/nsmc/raw/master/ratings.txt", filename="../data/ratings_train.txt")
df = pd.read_csv("../data/ratings_train.txt", sep = '\t').dropna()
df.head(5) # 긍정 : 1, 부정 : 0

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 199992 entries, 0 to 199999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        199992 non-null  int64 
 1   document  199992 non-null  object
 2   label     199992 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 6.1+ MB


In [4]:
X_train, X_test, y_train, y_test = train_test_split(df["document"], df["label"], random_state = 0)

# 문자 -> 숫자 벡터화
cnt_vec = CountVectorizer()
X_train_cnt = cnt_vec.fit_transform(X_train)

tfid = TfidfTransformer()
X_train_tfid = tfid.fit_transform(X_train_cnt)

# 모델 선언
model = MultinomialNB()
model.fit(X_train_tfid, y_train)

# 예측
y_pred = model.predict(cnt_vec.transform(X_test))
y_pred

array([1, 1, 0, ..., 1, 1, 0], dtype=int64)

In [5]:
print(classification_report(y_test, y_pred, target_names = ["Neg (0)", "Pos (1)"]))
# macro avg : 단순 평균
# weighted avg : 각 클래스에 속하는 표본의 갯수로 가중 평균
# accuracy : 정확도, 전체 학습 데이터의 갯수에서 각 클래스에서 자신의 클래스를 정확하게 맞춘 갯수의 비율

              precision    recall  f1-score   support

     Neg (0)       0.87      0.78      0.82     25050
     Pos (1)       0.80      0.88      0.84     24948

    accuracy                           0.83     49998
   macro avg       0.83      0.83      0.83     49998
weighted avg       0.83      0.83      0.83     49998



In [6]:
test = "강력한 몰입감을 선사하는 두 배우의 연기력!! 오랜만에 정말 색다른 영화였다"
result = model.predict(cnt_vec.transform([test]))
result_proba = model.predict_proba(cnt_vec.transform([test]))
result, result_proba

(array([1], dtype=int64), array([[7.09420303e-04, 9.99290580e-01]]))